# 03 — Exceções, validação e logging (padrão de projeto)

Objetivo: tratar erros do jeito certo em pipelines/análises e deixar rastros (logs).

Tempo: ~20–30 min

In [ ]:
from pathlib import Path

def find_repo_root(start: Path | None = None) -> Path:
    """Sobe diretórios até encontrar uma 'marca' do repositório (README.md + pasta data)."""
    cur = (start or Path.cwd()).resolve()
    for _ in range(10):
        if (cur / "README.md").exists() and (cur / "data").exists():
            return cur
        cur = cur.parent
    # fallback: assume cwd
    return Path.cwd().resolve()

ROOT = find_repo_root()
DATA_DIR = ROOT / "data"
SAMPLE_DIR = DATA_DIR / "sample"

print("ROOT:", ROOT)
print("SAMPLE_DIR:", SAMPLE_DIR)

## 1) Erros comuns em dados

- arquivo não existe
- coluna esperada não existe
- tipo errado (string onde deveria ser número)

A ideia aqui é falhar *rápido* e com mensagem útil.

In [ ]:
import pandas as pd

def load_sales(path):
    # Comentário importante: sempre valide o caminho antes de ler
    if not path.exists():
        raise FileNotFoundError(f"Arquivo não encontrado: {path}")
    df = pd.read_csv(path)

    # Validação mínima de esquema
    required = {"date","region","category","revenue"}
    missing = required - set(df.columns)
    if missing:
        raise ValueError(f"Colunas faltando: {sorted(missing)}")

    return df

sales = load_sales(SAMPLE_DIR / "sales.csv")
sales.head()

## 2) Logging

Use `logging` (não `print`) quando você quer histórico e debug mais fácil.

In [ ]:
import logging

logger = logging.getLogger("mentoria")
logger.setLevel(logging.INFO)

# Em notebooks, pode rolar duplicação de handlers; por isso o guard:
if not logger.handlers:
    handler = logging.StreamHandler()
    fmt = logging.Formatter("[%(levelname)s] %(message)s")
    handler.setFormatter(fmt)
    logger.addHandler(handler)

logger.info("Carregando sales.csv...")

In [ ]:
def safe_load_sales(path):
    try:
        df = load_sales(path)
        logger.info("OK: %s linhas, %s colunas", len(df), len(df.columns))
        return df
    except Exception as e:
        # Dica: em projetos reais, você pode logar stacktrace com logger.exception(...)
        logger.error("Falhou ao carregar: %s", e)
        return None

_ = safe_load_sales(SAMPLE_DIR / "sales.csv")
_ = safe_load_sales(SAMPLE_DIR / "nao_existe.csv")

## 3) Validação simples (antes de analisar)

Mini-checklist: nulos, tipos, duplicatas, faixas esperadas.

In [ ]:
import pandas as pd

df = sales.copy()
df["date"] = pd.to_datetime(df["date"], errors="coerce")

checks = {
    "nulos_total": int(df.isna().sum().sum()),
    "duplicadas_order_id": int(df.duplicated("order_id").sum()),
    "revenue_negativa": int((df["revenue"] < 0).sum()),
    "datas_invalidas": int(df["date"].isna().sum()),
}

checks

## Exercícios

1- Crie uma função `validate_positive(df, col)` que levanta erro se houver valores <= 0.
2- Faça `logger.exception(...)` dentro do except para capturar stacktrace.
3- Monte um dicionário `report` com 5 validações e salve em JSON.

In [ ]:
# Escreva suas respostas aqui
